In [44]:
import os
import pandas as pd

# 입력 파일 이름
#input_file_name = './data/chain-info.csv'
#input_file_name = './data/chain-market.csv'
#input_file_name = './data/chain-market-BTC.csv'
input_file_name = './result-files/market-chain.result.csv'

# Read the CSV file
df = pd.read_csv(input_file_name)

# ...
# invalid_rows = pd.to_datetime(df['_time'], errors='coerce').isna()
# print(df[invalid_rows])

def process_time(t):
    split_t = t.split('.')
    if len(split_t) > 1:
        if len(split_t[1]) == 1: 
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 2:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 3:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 4:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 5:
            return split_t[0] + '.000Z'
    else:  # If there is no decimal point
        return t[:-1] + '.000Z'

df['_time'] = df['_time'].apply(process_time)

df['_time'] = pd.to_datetime(df['_time'])

# Round the '_time' column to the nearest hour
df['_time'] = df['_time'].dt.round('H')

# Convert back to the original timestamp format without the timezone offset
# df['_time'] = df['_time'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

# Remove duplicate rows based on '_time' and 'chainName'
df = df.drop_duplicates(subset=['_time', 'chainName'])

# 파일의 기본 이름(디렉토리 없음)을 가져와서 파일 확장자 제거
base_file_name = os.path.basename(input_file_name)
base_file_name_without_extension = os.path.splitext(base_file_name)[0]

# 결과 파일이 저장될 디렉토리 정의
output_directory = './result-files/'

# 출력 파일 이름 생성
output_file_name = output_directory + base_file_name_without_extension + '.processed.csv'

# 누락된 값의 평균을 수정된 파일 이름의 텍스트 파일로 저장
df.to_csv(output_file_name, header=True)

In [43]:
import pandas as pd

# Step 1: Load each csv file into a separate DataFrame
chainInfo = pd.read_csv('./data/chain-info.csv')
chainMarket = pd.read_csv('./data/chain-market.csv')
chainMarketBTC = pd.read_csv('./data/chain-market-BTC.csv')

# Step 2: Select the required columns from each DataFrame
chainInfo = chainInfo[['apr', 'blockTime', 'bondedTokens', 'communityTokens', 'inflation', 'tps', '_time', 'chainName']]

# Add 'BTC' to each column name in chainMarketBTC
chainMarketBTC = chainMarketBTC[['cap', 'price', 'priceChangePercentage1H', 'totalVolume', '_time', 'chainName']]
chainMarketBTC.columns = [str(col) + 'BTC' if col not in ['_time', 'chainName'] else col for col in chainMarketBTC.columns]

# Step 3: Merge the DataFrames on '_time' and 'chainName' using inner merge
df = pd.merge(chainMarket, chainInfo, on=['_time', 'chainName'], how='inner')
df = pd.merge(df, chainMarketBTC, on=['_time', 'chainName'], how='inner')

# Step 4: Save the result as a new csv file
df.to_csv('./result-files/market-chain.result.csv', index=True)

